In [0]:
with cte_data (lines) as
(
select transform( filter(split(value, '\n'), (f,i) -> i % 2 = 0)
                , t -> filter( transform( split(t, '')
                                        , (u,i) -> struct(i, case u when 'S' then '|' else u end as v)
                                        )
                             , g -> g.v <> '.')
                )
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_07_input.txt', format => 'text', wholeText => true) as t (value)
/*from values ('.......S.......' || '\n'
           ||  '...............' || '\n'
           ||  '.......^.......' || '\n'
           ||  '...............' || '\n'
           ||  '......^.^......' || '\n'
           ||  '...............' || '\n'
           ||  '.....^.^.^.....' || '\n'
           ||  '...............' || '\n'
           ||  '....^.^...^....' || '\n'
           ||  '...............' || '\n'
           ||  '...^.^...^.^...' || '\n'
           ||  '...............' || '\n'
           ||  '..^...^.....^..' || '\n'
           ||  '...............' || '\n'
           ||  '.^.^.^.^.^...^.' || '\n'
           ||  '...............') as t (value)*/
)
  ,  cte_prep (tree) as
(
select aggregate( sequence(1, array_size(lines) - 1)
                , array(struct(0 as n, map_from_entries(transform(lines[0].i, x -> struct(x, 1l as s))) as p))
                , (acc, y) -> array_append( acc
                                          , struct( size(map_filter(element_at(acc, -1).p, (g,v) -> exists(lines[y], f -> g = f.i))) as n
                                                  , aggregate( array_sort(flatten(transform( map_entries(map_filter(element_at(acc, -1).p, (g,v) -> exists(lines[y], f -> g = f.i)))
                                                                                           , u -> array(struct(u.key - 1 as key, u.value), struct(u.key + 1 as key, u.value))))
                                                            || map_entries(map_filter(element_at(acc, -1).p, (g, v) -> not exists(lines[y], f -> g = f.i))))
                                                             , array()::array<struct<x:int,s:bigint>>
                                                             , (agg, z) -> case z.key
                                                                             when try_element_at(agg, -1).x
                                                                             then array_append(slice(agg, 1, array_size(agg) - 1), struct(element_at(agg, -1).x, element_at(agg, -1).s + z.value as s))
                                                                             else array_append(agg, struct(z.key as x, z.value as s))
                                                                           end
                                                             , agg -> map_from_entries(agg)
                                                             ) as p )
                                          )
                )
  from cte_data
)
select aggregate(tree.n                            , 0 , (agg, y) -> agg + y) as part1
     , aggregate(map_values(element_at(tree, -1).p), 0l, (agg, y) -> agg + y) as part2
  from cte_prep;
